<a href="https://colab.research.google.com/github/yohanesnuwara/reservoir-engineering/blob/master/Reservoir%20Simulation%20Ertekin/Unit%205%20Finite-Difference%20Approximation%20to%20Linear-Flow%20Equations/fd1d_linearflow_implicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Unit 5 Finite-Difference Approximations to Linear-Flow Problems**

## Implicit Formulation (*Backward-Difference Approximation*)

![Fig 5 30 Implicit Method](https://user-images.githubusercontent.com/51282928/76113827-60a28780-6017-11ea-9646-bb1c66683c08.PNG)

$$T_{lx_{i+1/2}}^{n} p_{i+1}^{n+1}-[(\frac{V_b \phi c_l}{\alpha_c B_l \Delta t})_{i} + T_{lx_{i+1/2}}^{n}+T_{lx_{i-1/2}}] \cdot p_{i}^{n+1} + T_{lx_{i-1/2}}^{n} p_{i-1}^{n+1} = -[q_{lsc_i}+(\frac{V_b \phi c_l}{\alpha_c B_l \Delta t})_{i} \cdot p_{i}^{n}]$$

In the following codes, $(\frac{V_b \phi c_l}{\alpha_c B_l \Delta t})_{i}$ will be written as `factor`

Transmissibility of coupling cells, $T_{lx_{i\pm1/2}}^{n}$, written as `T_min_half` or `T_plus_half`

The above equation will be simplified to:

$$pi_{next}^{+} \cdot p_{i+1}^{n+1}-pi_{next} \cdot p_{i}^{n+1} + pi_{next}^{-} \cdot p_{i-1}^{n+1} = pi$$

Where $pi_{next}^{+}$ (`pi_next_plus`), $pi_{next}$ (`pi_next`), $pi_{next}^{-}$ (`pi_next_minus`), and $pi$ (`pi`) are coefficients that contain the above `T_min_half` and `T_plus_half`.



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
!git clone https://github.com/yohanesnuwara/reservoir-engineering

Cloning into 'reservoir-engineering'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 909 (delta 91), reused 0 (delta 0), pack-reused 696
Receiving objects: 100% (909/909), 12.06 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (389/389), done.


## Example 5.11 Five reservoir gridblocks with no flow at both boundaries

![Example 5 8](https://user-images.githubusercontent.com/51282928/75949840-09e36380-5eda-11ea-824c-dd5c9ac6f6d5.PNG)

### For timestep 15 days

In [11]:
"Task. determine pressure distribution during first year of production with timestep 10 days"

# known
p_initial = 6000 # initial pressure, in psia
delta_x = 1000 # ft
delta_y = 1000
delta_z = 75
ngrid = 5
grid_loc = 4 # grid location where production well is located
B = 1 # phase FVF, assumed constant over pressure, rb/stb
c = 3.5E-06 # phase compressibility, psi^-1
k_x = 15 # perm in x direction, md
poro = 0.18
mu = 10 # phase viscosity, cp

delta_t = 15 # days
qsc = -150 # minus means production, stb/d

# conversion
k_x = 15 * 1E-03 # darcy to mD

# calculate factor
Vb = delta_x * delta_y * delta_z
alpha = 5.615 # volume conversion factor, is a constant
factor = (Vb * poro * c) / (alpha * B * delta_t)
factor

0.5609973285841495

Because of uniform gridblocks, the equation for `tr_plus_coupling` and `tr_min_coupling` become:

$$T_{lx_{i+1/2}}^{n} = T_{lx_{i-1/2}}^{n} = (\beta_c \frac{A_x k_x}{\mu_l B_l \Delta x})_{i+1/2}^{n} = (\beta_c \frac{A_x k_x}{\mu_l B_l \Delta x})_{i-1/2}^{n}$$

In [12]:
# calculate transmissibility of coupling cells
beta = 1.127 # transmissibility conversion factor, is a constant
Ax = delta_y * delta_z
T_plus_half = beta * ((Ax * k_x) / (mu * B * delta_x))
T_min_half = T_plus_half
T_min_half, T_plus_half

(0.1267875, 0.1267875)

There are 5 grids (`grid_1`, `grid_2`, `grid_3`, `grid_4`, `grid_5`), so the transmissibility coefficients will be: $$T_{lx_{i+1/2}}^{n}=(T_{lx_{1+1/2}}^{n}, T_{lx_{2+1/2}}^{n}, T_{lx_{3+1/2}}^{n}, T_{lx_{4+1/2}}^{n}, T_{lx_{5+1/2}}^{n})$$

and

$$T_{lx_{i-1/2}}^{n}=(T_{lx_{1/2}}^{n}, T_{lx_{1+1/2}}^{n}, T_{lx_{2+1/2}}^{n}, T_{lx_{3+1/2}}^{n}, T_{lx_{4+1/2}}^{n})$$

The values of $T_{lx_{5+1/2}}^{n}=0$ and $T_{lx_{1/2}}^{n}=0$ (the edge of gridblocks), so the values of each are:

$$T_{lx_{i+1/2}}^{n}=(0.127, 0.127, 0.127, 0.127, 0)$$

and

$$T_{lx_{i-1/2}}^{n}=(0, 0.127, 0.127, 0.127, 0.127)$$




In [0]:
q = np.full(ngrid-1, T_min_half)
Ti_plus_halves = np.append(q, [0])
print(Ti_plus_halves)
p = np.full(ngrid-1, T_plus_half)
Ti_min_halves = np.append([0], p)
print(Ti_min_halves)
print("At grid 1, the coupling transmissibility coeffs are:", Ti_min_halves[0], "for T_min_half and:", Ti_plus_halves[0], "for T_plus_half.")
print("At grid 3, the coupling transmissibility coeffs are:", Ti_min_halves[2], "for T_min_half and:", Ti_plus_halves[2], "for T_plus_half.")
print("At grid 5, the coupling transmissibility coeffs are:", Ti_min_halves[4], "for T_min_half and:", Ti_plus_halves[4], "for T_plus_half.")

[0.1267875 0.1267875 0.1267875 0.1267875 0.       ]
[0.        0.1267875 0.1267875 0.1267875 0.1267875]
At grid 1, the coupling transmissibility coeffs are: 0.0 for T_min_half and: 0.1267875 for T_plus_half.
At grid 3, the coupling transmissibility coeffs are: 0.1267875 for T_min_half and: 0.1267875 for T_plus_half.
At grid 5, the coupling transmissibility coeffs are: 0.1267875 for T_min_half and: 0.0 for T_plus_half.


Requires array for $q_{sc}$

In [0]:
qsc = [0, 0, 0, -150, 0] # production well in grid 4

Calculate $p_{i}^{n+1}$ for each grid in each time

Steps in iteration:
* Calculate coefficients for `pi_next_minus`, `pi_next`, `pi_next_plus`, and `pi` for each grid in each timestep

* Construct tridiagonal matrix from the calculated `pi_next_minus`, `pi_next`, `pi_next_plus`, and `pi` in each timestep

* Solve the tridiagonal matrix with **TDMA** in each timestep

### For the first timestep

In [63]:
import sys, os
sys.path.append('/content/reservoir-engineering/Reservoir Simulation Ertekin/Unit 5 Finite-Difference Approximation to Linear-Flow Equations')
from tdma_implicit import TDMAsolver

pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]

pi_next_plus_arr = []
pi_next_arr = []
pi_next_minus_arr = []
pi_arr = []

for i in range(len(pi)):
  pi_next_plus = Ti_plus_halves[i]
  pi_next = -(factor + Ti_plus_halves[i] + Ti_min_halves[i])
  pi_next_minus = Ti_min_halves[i]
  pi_current = -(qsc[i] + (factor * pi[i]))

  pi_next_plus_arr.append(float(pi_next_plus))
  pi_next_arr.append(float(pi_next))
  pi_next_minus_arr.append(float(pi_next_minus))
  pi_arr.append(float(pi_current))

pi_next_plus_arr = pi_next_plus_arr[:-1]
pi_next_minus_arr = pi_next_minus_arr[1:]

p_new = TDMAsolver(pi_next_minus_arr, pi_next_arr, pi_next_plus_arr, pi_arr)
p_new

array([5999.08253536, 5995.02302466, 5968.94186457, 5805.43760672,
       5964.13401632])

### For the next timesteps (iterative)

In [76]:
import sys, os
sys.path.append('/content/reservoir-engineering/Reservoir Simulation Ertekin/Unit 5 Finite-Difference Approximation to Linear-Flow Equations')
from tdma_implicit import TDMAsolver

pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]
time = np.arange(15, 370, delta_t)

p_new_arr = []

for j in range(len(time)):
  pi_next_plus_arr = []
  pi_next_arr = []
  pi_next_minus_arr = []
  pi_arr = []

  for i in range(len(pi)):
    pi_next_plus = Ti_plus_halves[i]
    pi_next = -(factor + Ti_plus_halves[i] + Ti_min_halves[i])
    pi_next_minus = Ti_min_halves[i]
    pi_current = -(qsc[i] + (factor * pi[i]))

    pi_next_plus_arr.append(float(pi_next_plus))
    pi_next_arr.append(float(pi_next))
    pi_next_minus_arr.append(float(pi_next_minus))
    pi_arr.append(float(pi_current))

  pi_next_plus_arr = pi_next_plus_arr[:-1]
  pi_next_minus_arr = pi_next_minus_arr[1:]

  p_new = TDMAsolver(pi_next_minus_arr, pi_next_arr, pi_next_plus_arr, pi_arr)
  p_new_arr.append(p_new)
  pi = p_new # updating pi

df = pd.DataFrame.from_records(p_new_arr)
df = pd.DataFrame(pd.np.column_stack([time, df]), columns=['time', 'grid 1', 'grid 2', 'grid 3', 'grid 4', 'grid 5'])
df

,time,grid 1,grid 2,grid 3,grid 4,grid 5
0,15.0,5999.082535,5995.023025,5968.941865,5805.437607,5964.134016
1,30.0,5996.288586,5983.926184,5922.463533,5655.347957,5907.211835
2,45.0,5990.905854,5967.088850,5868.771602,5532.883385,5838.207451
3,60.0,5982.510213,5945.361975,5812.078717,5427.946146,5762.579140
4,75.0,5970.918387,5919.627971,5754.472173,5334.424389,5683.652318
5,90.0,5956.120544,5890.644450,5696.924816,5248.575083,5603.449393
6,105.0,5938.219314,5859.011646,5639.838141,5168.072925,5523.191307
7,120.0,5917.382850,5825.187636,5583.331141,5091.447822,5443.602932
8,135.0,5893.811872,5789.517243,5527.391637,5017.750206,5365.100470
9,150.0,5867.718063,5752.260646,5471.953765,4946.350635,5287.907367
